In [1]:
import torch
import torch.onnx
import torch.nn as nn
import torch.optim as optim
import onnx
import torch.nn.functional as F
import tensorflow as tf
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import kagglehub as

In [2]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: kagglehub in c:\users\karthik\appdata\local\programs\python\python310\lib\site-packages (0.3.12)




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install onnx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip list

Package                      Version
---------------------------- -----------
3                            0.0.0
absl-py                      2.2.2
altair                       5.5.0
asttokens                    3.0.0
astunparse                   1.6.3
attrs                        25.3.0
blinker                      1.9.0
cachetools                   5.5.2
certifi                      2025.1.31
cffi                         1.17.1
charset-normalizer           3.4.1
click                        8.1.8
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.3.2
cv                           1.0.0
cycler                       0.12.1
debugpy                      1.8.14
decorator                    5.2.1
exceptiongroup               1.2.2
executing                    2.2.0
filelock                     3.18.0
flatbuffers                  25.2.10
fonttools                    4.57.0
fsspec                       2025.3.2
gast                         0.4

In [11]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset",force_download=True)

print("Path to dataset files:", path)

Resuming download from 1894776832 bytes (1002932355 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/vipoooool/new-plant-diseases-dataset?dataset_version_number=2 (1894776832/2897709187) bytes left.


100%|██████████| 2.70G/2.70G [05:19<00:00, 3.14MB/s]

Extracting files...


Path to dataset files: C:\Users\Karthik\.cache\kagglehub\datasets\vipoooool\new-plant-diseases-dataset\versions\2


In [12]:
# # The path to the main directory 
# main_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'  

# # List all classes in the main directory
# classes = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]


# The correct path to the main directory on your local machine
# We need to append the subdirectories based on the dataset's structure
main_dir = os.path.join(path, 'New Plant Diseases Dataset(Augmented)', 'New Plant Diseases Dataset(Augmented)', 'train')

# List all classes in the main directory
classes = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]
# Print the names of the classes
print("\nClasses:")
for class_name in classes:
    print(class_name)


Classes:
Apple___Apple_scab
Apple___Black_rot
Apple___Cedar_apple_rust
Apple___healthy
Blueberry___healthy
Cherry_(including_sour)___healthy
Cherry_(including_sour)___Powdery_mildew
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Common_rust_
Corn_(maize)___healthy
Corn_(maize)___Northern_Leaf_Blight
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___healthy
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Orange___Haunglongbing_(Citrus_greening)
Peach___Bacterial_spot
Peach___healthy
Pepper,_bell___Bacterial_spot
Pepper,_bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Raspberry___healthy
Soybean___healthy
Squash___Powdery_mildew
Strawberry___healthy
Strawberry___Leaf_scorch
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___healthy
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_mosaic_virus
Tomato___Tomato_Yellow_Leaf_Curl_Virus


In [13]:
# # List the classes to work on
# # subdirs_to_copy = ['Grape___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___Black_rot','Corn_(maize)___Common_rust_','Grape___Esca_(Black_Measles)','Corn_(maize)___healthy']  
# subdirs_to_copy = classes
# # Create a new directory in the Kaggle working directory
# subset_dir = '/kaggle/working/new_directory'
# os.makedirs(subset_dir, exist_ok=True)

# # Copy the selected classes to the new directory
# for subdir_name in subdirs_to_copy:
#     src_dir_path = os.path.join(main_dir, subdir_name)
#     dest_dir_path = os.path.join(subset_dir, subdir_name)
#     shutil.copytree(src_dir_path, dest_dir_path)

# print(f"Subdirectories {subdirs_to_copy} copied to {subset_dir}")




# List the classes to work on
subdirs_to_copy = classes # Using all detected classes as per your request

# Create a new directory in the current working directory for the subset
# This will create a 'plant_diseases_subset' folder where you run this script
subset_dir = os.path.join(os.getcwd(), 'plant_diseases_subset')
os.makedirs(subset_dir, exist_ok=True) # Use exist_ok=True to avoid error if directory already exists

# Copy the selected classes to the new directory
print(f"\nCopying selected subdirectories to: {subset_dir}")
for subdir_name in subdirs_to_copy:
    src_dir_path = os.path.join(main_dir, subdir_name)
    dest_dir_path = os.path.join(subset_dir, subdir_name)

    # Check if the source directory exists before attempting to copy
    if os.path.exists(src_dir_path) and os.path.isdir(src_dir_path):
        # Remove destination if it exists to ensure a clean copy, especially if force_download was used
        if os.path.exists(dest_dir_path):
            shutil.rmtree(dest_dir_path)
        shutil.copytree(src_dir_path, dest_dir_path)
        print(f"Copied '{subdir_name}'")
    else:
        print(f"Warning: Source directory '{src_dir_path}' not found. Skipping.")


print(f"\nSubdirectories {subdirs_to_copy} copied to {subset_dir}")
print(f"You can find the copied data at: {subset_dir}")


Copying selected subdirectories to: d:\Capstone_MiniProjects_Guvi\Final_Pros\Plant-Disease-Detection-from-Images-main\Plant-Disease-Detection-from-Images-main\plant_diseases_subset
Copied 'Apple___Apple_scab'
Copied 'Apple___Black_rot'
Copied 'Apple___Cedar_apple_rust'
Copied 'Apple___healthy'
Copied 'Blueberry___healthy'
Copied 'Cherry_(including_sour)___healthy'
Copied 'Cherry_(including_sour)___Powdery_mildew'
Copied 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'
Copied 'Corn_(maize)___Common_rust_'
Copied 'Corn_(maize)___healthy'
Copied 'Corn_(maize)___Northern_Leaf_Blight'
Copied 'Grape___Black_rot'
Copied 'Grape___Esca_(Black_Measles)'
Copied 'Grape___healthy'
Copied 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)'
Copied 'Orange___Haunglongbing_(Citrus_greening)'
Copied 'Peach___Bacterial_spot'
Copied 'Peach___healthy'
Copied 'Pepper,_bell___Bacterial_spot'
Copied 'Pepper,_bell___healthy'
Copied 'Potato___Early_blight'
Copied 'Potato___healthy'
Copied 'Potato___Late_blight'
C

In [14]:
# Initialize a counter for the number of images
image_count = 0

# Iterate over all classes and count the images
for subdir, _, files in os.walk(subset_dir):
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]
    image_count += len(image_files)

print(f"Total number of images: {image_count}")

Total number of images: 70295


### data preprocessing

In [15]:
#dataset_path = '/kaggle/working/new_directory'
dataset_path = 'd:\Capstone_MiniProjects_Guvi\Final_Pros\Plant-Disease-Detection-from-Images-main\Plant-Disease-Detection-from-Images-main\plant_diseases_subset'

In [16]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),              # resize all images to 150x150 pixels
    transforms.ToTensor(),                      # convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [17]:
dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [18]:
dataset.classes

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

### train test split 


In [19]:
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=0.3, random_state=42)

train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

In [20]:
trainloader = DataLoader(train_dataset, batch_size=15, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=15, shuffle=True)

### define cnn model

In [21]:
class CNN_Classification(nn.Module):
    def __init__(self):
        super(CNN_Classification, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)       # first convolutional layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)            # first pooling layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)      # second convolutional layer
        self.fc1 = nn.Linear(64 * 37 * 37, 128)                                 # 37 x 37 is the size after pooling
        self.fc2 = nn.Linear(128, len(dataset.classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)                                                 # flatten all dimensions
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [22]:
CNN_Model = CNN_Classification()

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_Model.parameters(), lr=0.001)          # Defines the optimizer, Adam with a learning rate of 0.001

### train the model

In [24]:
num_epochs = 5
for epoch in range(num_epochs):

    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = CNN_Model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


print('Finished Training')

Epoch 1/5, Loss: 0.8420367240905762
Epoch 2/5, Loss: 0.13923266530036926
Epoch 3/5, Loss: 0.007987145334482193
Epoch 4/5, Loss: 0.010538074187934399
Epoch 5/5, Loss: 0.0013543072855100036
Finished Training


### evaluation

In [25]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation mode
CNN_Model.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = CNN_Model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.4f}%')
print(f'Recall: {recall * 100:.4f}%')
print(f'F1-score: {f1 * 100:.4f}%')

Accuracy: 87.46 %
Precision: 87.8696%
Recall: 87.4579%
F1-score: 87.3738%


### Prediction


In [43]:
# Load a test image and preprocess it
img = Image.open('D:/Capstone_MiniProjects_Guvi/Final_Pros/Plant-Disease-Detection-from-Images-main/Plant-Disease-Detection-from-Images-main/plant_diseases_subset/Apple___Cedar_apple_rust/0a41c25a-f9a6-4c34-8e5c-7f89a6ac4c40___FREC_C.Rust 9807.jpg')
img = transform(img).unsqueeze(0)  # add batch dimension

# Pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)
_, predicted = torch.max(output, 1)
print(f'Predicted class: {dataset.classes[predicted.item()]}')

Predicted class: Apple___Cedar_apple_rust


In [45]:
# Load a test image and preprocess it
img = Image.open('D:/Capstone_MiniProjects_Guvi/Final_Pros/Plant-Disease-Detection-from-Images-main/Plant-Disease-Detection-from-Images-main/plant_diseases_subset/Apple___Cedar_apple_rust/0a41c25a-f9a6-4c34-8e5c-7f89a6ac4c40___FREC_C.Rust 9807_90deg.JPG')
img = transform(img).unsqueeze(0)  # add batch dimension

# Pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)
_, predicted = torch.max(output, 1)
print(f'Predicted class: {dataset.classes[predicted.item()]}')

Predicted class: Apple___Cedar_apple_rust


In [46]:
torch.save(CNN_Model.state_dict(), 'cnn_model.pth')  

In [48]:
CNN_Model.load_state_dict(torch.load('D:\Capstone_MiniProjects_Guvi\Final_Pros\Plant-Disease-Detection-from-Images-main\Plant-Disease-Detection-from-Images-main\cnn_model.pth'))

<All keys matched successfully>

### Pretrained models

### Vgg16

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Transfer model to the GPU
vgg16 = vgg16.to(device)

# Freeze all the layers (optional, if you don't want to train the convolutional layers)
for param in vgg16.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
vgg16.classifier[6] = nn.Linear(4096, len(dataset.classes))  # 4096 is the input to the last layer

# Transfer classifier changes to the GPU
vgg16.classifier = vgg16.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move criterion to GPU
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = vgg16(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training VGG16')

Using device: cpu


c:\Users\Karthik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Karthik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Karthik/.cache\torch\hub\checkpoints\vgg16-397923af.pth


100%|██████████| 528M/528M [02:48<00:00, 3.29MB/s] 


Epoch 1/5, Loss: 1.0158711671829224
Epoch 2/5, Loss: 1.1700907945632935
Epoch 3/5, Loss: 0.016897844150662422
Epoch 4/5, Loss: 2.289205551147461
Epoch 5/5, Loss: 0.61502605676651
Finished Training VGG16


In [50]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation mode
vgg16.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = vgg16(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.4f}%')
print(f'Recall: {recall * 100:.4f}%')
print(f'F1-score: {f1 * 100:.4f}%')

Accuracy: 85.64 %
Precision: 86.6324%
Recall: 85.6418%
F1-score: 85.3993%


### AlexNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Transfer model to the GPU
alexnet = alexnet.to(device)

# Freeze layers if desired
for param in alexnet.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
alexnet.classifier[6] = nn.Linear(4096, len(dataset.classes))

# Transfer classifier changes to the GPU
alexnet.classifier = alexnet.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move loss function to GPU
optimizer = optim.Adam(alexnet.parameters(), lr=0.02)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training AlexNet')

Using device: cpu


c:\Users\Karthik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Karthik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\Karthik/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [01:13<00:00, 3.33MB/s] 


Epoch 1/5, Loss: 43.58063888549805
Epoch 2/5, Loss: 10.098589897155762
Epoch 3/5, Loss: 14.871909141540527
Epoch 4/5, Loss: 46.39433288574219
Epoch 5/5, Loss: 5.143483638763428
Finished Training AlexNet


In [52]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation model
alexnet.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = alexnet(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1-score: {f1 * 100:.2f}%')

Accuracy: 87.51 %
Precision: 88.20%
Recall: 87.51%
F1-score: 87.38%


### DenseNet

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained DenseNet121 model
densenet = models.densenet121(pretrained=True)

# Transfer model to the GPU
densenet = densenet.to(device)

# Freeze all the layers if you don't want to train the convolutional layers (optional)
for param in densenet.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
num_ftrs = densenet.classifier.in_features
densenet.classifier = nn.Linear(num_ftrs, len(dataset.classes))

# Transfer classifier changes to the GPU
densenet.classifier = densenet.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move loss function to GPU
optimizer = optim.Adam(densenet.parameters(), lr=0.02)

# Train the model
num_epochs = 4
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = densenet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training DenseNet')

Using device: cpu
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\Karthik/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


c:\Users\Karthik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Karthik\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 30.8M/30.8M [00:09<00:00, 3.53MB/s]


Epoch 1/4, Loss: 3.1420345306396484
Epoch 2/4, Loss: 0.5805450677871704
Epoch 3/4, Loss: 0.0
Epoch 4/4, Loss: 7.964588642120361
Finished Training DenseNet


In [54]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation model
# densenet.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = densenet(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1-score: {f1 * 100:.2f}%')


Accuracy: 88.27 %
Precision: 89.99%
Recall: 88.27%
F1-score: 88.21%


In [55]:
torch.save(vgg16.state_dict(), 'vgg16.pth')  
torch.save(alexnet.state_dict(), 'alexnet_model.pth') 
torch.save(densenet.state_dict(), 'densenet.pth')